# Logistic regression plotly example

Made by Samuel Kekäläinen, based on code by Johan Vestö

Source:
https://github.com/NoviaIntSysGroup/resources-and-learning-material/blob/main/Interactive%20demos/LogisticRegression.py

#### TODO
 * ~~Fix bars at the bottom to be on one row~~
 * ~~Create buttons for training etc.~~
 * ~~Add actual plot~~
 * ~~Make UI~~
 * Remake the path method used in reference app
 * Add the actual formula to the graph and not just the same every time
 * Implement learning button
 * Implement reset button

In [1]:
%pip install numpy
%pip install dash
%pip install plotly
%pip install pandas
# Install libraries in order to run the baseline
%pip install matplotlib
#%pip install tkinter


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas # Apparently necessary for Plotly
#import tkinter as tk
#from matplotlib import cm
#from matplotlib.figure import Figure
#from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)

import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go


### Defining parameters

In [3]:
# Parameters
mu1 = -1		    # mu for class 1 # mu = mean?
mu2 = 2		   	 	# mu for class 2
sigma = 1	    	# std around mu
n = 30				# Number of data points
x_lim = [mu1-3, mu2+3] 	# +- limits for generated x-values

# Resolution and limits for the error surface
dW = 0.05
W0_lim = (-6, 4) 
W1_lim = (-1, 6)



### Generating data and arrays

In [4]:
# Generating data points
x_class1 =  sigma*np.random.randn(n, 1) + mu1
x_class2 =  sigma*np.random.randn(n, 1) + mu2
x = np.vstack([x_class1, x_class2])
X = np.hstack([np.ones([2*n, 1]), x])
y = np.vstack([np.zeros([n, 1]), np.ones([n, 1])])

# Generate linearly spaced x-values for model predictions
x_model = np.linspace(x_lim[0], x_lim[1], 101)				# Linearly spaced x-values
X_model = np.vstack( (np.ones(x_model.size), x_model) ).T 	# X matrix, [1, x]

# Creating matrices for computing the error surface
W = np.random.rand(2).reshape(1,2)			# Initial random guesss
W0 = np.arange(W0_lim[0],W0_lim[1]+dW,dW)	# Array of w0 values
W1 = np.arange(W1_lim[0],W1_lim[1]+dW,dW)	# Array of w1 values
W0, W1 = np.meshgrid(W0,W1)					# W0 and W1 matrices
nll = np.zeros(W0.size).reshape(W0.shape)	# Empty matrix for nll
# Average negative loglikelihood
i_max, j_max = nll.shape
for i in range(i_max):
	for j in range(j_max):
		W_tmp = np.array( [ W0[i,j], W1[i,j] ] ).reshape(1,2)	# Weight combination
		z = np.dot( W_tmp, X.T )		# Similarity score
		y_hat = 1 / (1 + np.exp(-z))	# Model outputs
		ll = y*np.log(y_hat.T) + (1-y)*np.log(1-y_hat.T)
		nll[i,j] = -ll.mean()

### Create app

In [5]:
# Create Dash app
app = dash.Dash(__name__)
app.title = "Logistic regression plotly example"

# Define layout
app.layout = html.Div([
    html.H1("Logistic Regression Interactive Demo", style={'color': 'blue'}),  # Change color to blue

    # Figures side by side
    html.Div([
        dcc.Graph(
            id='data-and-model',
            figure={
                'data': [
                    {'x': x[y == 0], 'y': y[y == 0], 'mode': 'markers', 'name': 'Class 0'},
                    {'x': x[y == 1], 'y': y[y == 1], 'mode': 'markers', 'name': 'Class 1'},
                    {'x': x_model, 'y': [], 'mode': 'lines', 'name': 'Model'},
                ],
                'layout': {
                    'xaxis': {'title': 'x'},
                    'yaxis': {'title': 'y; p(y=1)', 'range': [-0.1, 1.1]},
                    'title': 'Data and model',
                    'legend': {'x': 1.05, 'y': 0.5},
                }
            }
        ),
        dcc.Graph(
            id='objective-function',
            figure={
                'data': [go.Contour(x=W0[0], y=W1[:, 0], z=nll)],
                'layout': {
                    'xaxis': {'title': '$w_0$'},
                    'yaxis': {'title': '$w_1$'},
                    'title': 'Objective function',
                }
            }
        ),
    ], style={'display': 'flex'}),

    # Title for sliders
    #html.Div([
    #    html.H3("Slider Controls", style={'color': 'green'}),  # Change color to green
    #]),

    # Sliders horizontally
    html.Div([
        html.Div([
            html.H4("Bias", style={'color': 'red'}),  # Change color to red
            dcc.Slider(
                id='bias-slider',
                min=W0_lim[0],
                max=W0_lim[1],
                step=0.1,
                value=0.0,
                marks={val: str(val) for val in range(W0_lim[0], W0_lim[1] + 1)},
                tooltip={'placement': 'bottom', 'always_visible': True},
            ),
        ], style={'width': '30%', 'margin-right': '5%'}),
        html.Div([
            html.H4("Slope", style={'color': 'purple'}),  # Change color to purple
            dcc.Slider(
                id='slope-slider',
                min=W1_lim[0],
                max=W1_lim[1],
                step=0.1,
                value=0.0,
                marks={val: str(val) for val in range(W1_lim[0], W1_lim[1] + 1)},
                tooltip={'placement': 'bottom', 'always_visible': True},
            ),
        ], style={'width': '30%', 'margin-right': '5%'}),
        html.Div([
            html.H4("Learning Rate", style={'color': 'orange'}),  # Change color to orange
            dcc.Slider(
                id='learning-rate-slider',
                min=0.2,
                max=4.0,
                step=0.2,
                value=1.0,
                marks={val: str(val) for val in np.arange(0.2, 4.1, 0.2)},
                tooltip={'placement': 'bottom', 'always_visible': True},
            ),
        ], style={'width': '30%'}),
    ], style={'display': 'flex', 'justify-content': 'space-between'}),
    html.Button('Reset Model', id='reset-button', n_clicks=0, style={'margin': '10px'}),
    html.Button('Training Step', id='step-button', n_clicks=0, style={'margin': '10px'}),
])


### Functions

In [ ]:
# Functions from original app, redone

def updateModelPrediction(*args):
	# Model Prediction
	
    # z is used as the exponent, and is what creates the value. 
	# The W[-1, : ] part is just used to pull it out properly
	# W is size (1,2) and X_model is size (101, 2). So I guess bias is the one acting on the array of 1's, and slope on the x-values? 
    z = np.dot( W[-1, :], X_model.T ).flatten()
    # W = array of bias + slope. Bias is the first one, slope the second one?
	# X_model = [[1, x0], [1, x1], ...]
	# There is both X_model and x_model... from definition:
	
    # Generate linearly spaced x-values for model predictions
    #x_model = np.linspace(x_lim[0], x_lim[1], 101)				# Linearly spaced x-values
    #X_model = np.vstack( (np.ones(x_model.size), x_model) ).T 	# X matrix, [1, x] # Seems to be a bunch of 1's with the appropriate X-value afterwards. I guess the 1's are manipulated to get the final result?

    y_hat = 1 / (1 + np.exp(-z))
	# Update model
	#model.set_data(x_model, y_hat) # model = matplotlib object of some sort
    # x_model = the x-values that are "plotted"
	# y_hat = value predicted by model

    model_text = r'$f(x) = 1/(1+\exp(-{:1.1f}x - {:1.1f})$'.format(W[-1, 1], W[-1, 0])
	
	#text.set_text(model_text)
	#canvas.draw()

def updatePath():
	# Model Prediction
	z = np.dot( W[-1, :], X.T )
	y_hat = 1 / (1 + np.exp(-z))
	# Gradient
	e = y - y_hat[:, np.newaxis]
	dW = 1 / n * np.dot( e.T, X )
	dW = dW / np.linalg.norm(dW)
	# Update path
	path.set_data(W[:, 0], W[:, 1])
	ax2.findobj(lambda artist : artist.get_gid() == 'arrow')[0].remove()
	ax2.quiver(W[-1, 0], W[-1, 1], dW[0, 0], dW[0, 1], scale_units='inches', scale=3, gid='arrow')
	canvas.draw()

def updateWeights(*args):
	global W
	W = W[0, :].reshape(1, 2)
	W[0, 0] = slider_bias.get()
	W[0, 1] = slider_slope.get()
	updateModelPrediction()
	updatePath()

def resetPath(*args):
	global W
	W = W[0, :].reshape(1, 2)
	updateModelPrediction()
	updatePath()

def takeLearningStep():
	global W
	eta = slider_eta.get()				# Read out the learning rate
	z = np.dot( W[-1, :], X.T )			# Similarity score
	y_hat = 1 / (1 + np.exp(-z))
	e = y - y_hat[:, np.newaxis]		# error signal
	dW = - eta / n * np.dot( e.T, X )	# Gradient
	W = np.append(W, W[-1, :] - dW, 0)	# Take a step
	updateModelPrediction()
	updatePath()

### Callback functions

In [6]:
# Define callback to update model prediction, path, and pointer
@app.callback(
    [Output('data-and-model', 'figure'),
     Output('objective-function', 'figure')],
    [Input('bias-slider', 'value'),
     Input('slope-slider', 'value'),
     Input('learning-rate-slider', 'value')]
)
def update_figures(bias, slope, learning_rate):
    global X_model, X, y, W

    # Model Prediction
    z = np.dot([bias, slope], X_model.T).flatten()
    y_hat = 1 / (1 + np.exp(-z))

    # Update model trace
    data_and_model_figure = {
        'data': [
            {'x': x[y == 0], 'y': y[y == 0], 'mode': 'markers', 'name': 'Class 0'},
            {'x': x[y == 1], 'y': y[y == 1], 'mode': 'markers', 'name': 'Class 1'},
            {'x': x_model, 'y': y_hat, 'mode': 'lines', 'name': 'Model'},
        ],
        'layout': {
            'xaxis': {'title': 'x'},
            'yaxis': {'title': 'y; p(y=1)', 'range': [-0.1, 1.1]},
            'title': 'Data and model',
            'legend': {'x': 1.05, 'y': 0.5},
        }
    }

    # Update objective function figure
    W = np.array([bias, slope]).reshape(1, 2)
    nll_surface = np.zeros(W0.shape)
    for i in range(W0.shape[0]):
        for j in range(W0.shape[1]):
            W_tmp = np.array([W0[i, j], W1[i, j]])
            z = np.dot(W_tmp, X.T)
            y_hat = 1 / (1 + np.exp(-z))
            ll = y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat)
            nll_surface[i, j] = -ll.mean()

    # Create a scatter plot with a single point for the current position
    pointer_figure = {
        'data': [go.Scatter(
            x=[bias],
            y=[slope],
            mode='markers',
            marker=dict(color='red', size=10),
            showlegend=False
        )],
        'layout': {
            'xaxis': {'title': '$w_0$'},
            'yaxis': {'title': '$w_1$'},
            'title': 'Objective function with Current Position',
            'margin': {'l': 10, 'r': 10, 't': 30, 'b': 10},
            'autosize': True,
        }
    }

    # Include the pointer in the layout of the objective function figure
    objective_function_figure = {
        'data': [go.Contour(x=W0[0], y=W1[:, 0], z=nll_surface)],
        'layout': {
            'xaxis': {'title': '$w_0$'},
            'yaxis': {'title': '$w_1$'},
            'title': 'Objective function',
            'shapes': [
                {
                    'type': 'circle',
                    'xref': 'x',
                    'yref': 'y',
                    'x0': bias - 0.1,
                    'y0': slope - 0.1,
                    'x1': bias + 0.1,
                    'y1': slope + 0.1,
                    'line': {
                        'color': 'red',
                    },
                },
            ],
        }
    }

    return data_and_model_figure, objective_function_figure


cmnt = '''
# Add callback for both buttons and sliders
@app.callback(
    [Output('data-and-model', 'figure'),
     Output('objective-function', 'figure')],
    [Input('reset-button', 'n_clicks'),
     Input('step-button', 'n_clicks')],
    [dash.dependencies.State('bias-slider', 'value'),
     dash.dependencies.State('slope-slider', 'value'),
     dash.dependencies.State('learning-rate-slider', 'value'),
     dash.dependencies.State('learning-rate-input', 'value')]
)
def update_figures(reset_clicks, step_clicks, bias, slope, learning_rate_slider, learning_rate_input):
    global X_model, X, y, W

    if reset_clicks > 0:
        # Reset button logic
        # Update X_model, X, y, and W accordingly

        # Example:
        # W = initial_weights
        # Update X_model, X, y, and W accordingly

    if step_clicks > 0:
        # Training step button logic
        # Use learning_rate_slider or learning_rate_input for the learning rate
        # Update X_model, X, y, and W accordingly

        # Example:
        #W = W - learning_rate * gradient
        W = calc_learning_step(learning_rate_input)

    # Model Prediction
    z = np.dot([bias, slope], X_model.T).flatten()
    y_hat = 1 / (1 + np.exp(-z))

    # Update model trace
    data_and_model_figure = {
        'data': [
            {'x': x[y == 0], 'y': y[y == 0], 'mode': 'markers', 'name': 'Class 0'},
            {'x': x[y == 1], 'y': y[y == 1], 'mode': 'markers', 'name': 'Class 1'},
            {'x': x_model, 'y': y_hat, 'mode': 'lines', 'name': 'Model'},
        ],
        'layout': {
            'xaxis': {'title': 'x'},
            'yaxis': {'title': 'y; p(y=1)', 'range': [-0.1, 1.1]},
            'title': 'Data and model',
            'legend': {'x': 1.05, 'y': 0.5},
        }
    }

    # Update objective function figure
    W = np.array([bias, slope]).reshape(1, 2)
    nll_surface = np.zeros(W0.shape)
    for i in range(W0.shape[0]):
        for j in range(W0.shape[1]):
            W_tmp = np.array([W0[i, j], W1[i, j]])
            z = np.dot(W_tmp, X.T)
            y_hat = 1 / (1 + np.exp(-z))
            ll = y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat)
            nll_surface[i, j] = -ll.mean()

    # Include the pointer in the layout of the objective function figure
    objective_function_figure = {
        'data': [go.Contour(x=W0[0], y=W1[:, 0], z=nll_surface)],
        'layout': {
            'xaxis': {'title': '$w_0$'},
            'yaxis': {'title': '$w_1$'},
            'title': 'Objective function',
            'shapes': [
                {
                    'type': 'circle',
                    'xref': 'x',
                    'yref': 'y',
                    'x0': bias - 0.1,
                    'y0': slope - 0.1,
                    'x1': bias + 0.1,
                    'y1': slope + 0.1,
                    'line': {
                        'color': 'red',
                    },
                },
            ],
        }
    }


    return data_and_model_figure, objective_function_figure



def calc_learning_step(learning_rate):
    global W
    eta = learning_rate              # Read out the learning rate
    z = np.dot( W[-1, :], X.T )         # Similarity score
    y_hat = 1 / (1 + np.exp(-z))
    e = y - y_hat[:, np.newaxis]        # error signal
    dW = - eta / n * np.dot( e.T, X )   # Gradient
    W = np.append(W, W[-1, :] - dW, 0)  # Take a step
    return W[-1] # Get the latest ?
'''

# NOTE: UNIMPLEMENTED
def takeLearningStep():
    global W
    eta = slider_eta.get()              # Read out the learning rate
    z = np.dot( W[-1, :], X.T )         # Similarity score
    y_hat = 1 / (1 + np.exp(-z))
    e = y - y_hat[:, np.newaxis]        # error signal
    dW = - eta / n * np.dot( e.T, X )   # Gradient
    W = np.append(W, W[-1, :] - dW, 0)  # Take a step
    updateModelPrediction()
    updatePath()


In [7]:
#print(x_model, y_hat)

### Running the server

In [8]:
app.run_server(mode='inline')